<h2>TODO: Connect neo4j with python</h2>

In [1]:
from neo4j import GraphDatabase
import pandas as pd

In [2]:
# class HelloWorldExample:

#     def __init__(self, uri, user, password):
#         self.driver = GraphDatabase.driver(uri, auth=(user, password))

#     def close(self):
#         self.driver.close()

#     def print_greeting(self, message):
#         with self.driver.session() as session:
#             greeting = session.write_transaction(self._create_and_return_greeting, message)
#             print(greeting)

#     @staticmethod
#     def _create_and_return_greeting(tx, message):
#         result = tx.run("CREATE (a:Greeting) "
#                         "SET a.message = $message "
#                         "RETURN a.message + ', from node ' + id(a)", message=message)
#         return result.single()[0]


# if __name__ == "__main__":
#     greeter = HelloWorldExample("bolt://localhost:7687", "neo4j", "fighting1")
#     greeter.print_greeting("hello, world")
#     greeter.close()

<h2>Build dataset for Neo4j Desktop</h2>

<h3>Entities: case, plaintiff, defendant, lawyer, court, judge</h3>
<h3>Relationships(directed):<br> 
    case->plaintiff: plaintiff, <br>
<!--     plaintiff->case: launch, <br> -->
    case->defendant: defendant, <br>
    plaintiff->defendant: sue, <br>
<!--     defendant->plaintiff: be sued, <br> -->
<!--     defendant->case: be sued, <br> -->
    plaintiff->lawyer: authorize, <br>
    case->court: locate, <br>
    case->judge: judge, <br>
    case->lawyer: lawyer, <br>
    judge->court: work in <br>
</h3>

In [3]:
all_cases = pd.read_csv('/Users/starice/OwnFiles/cityu/RA/case_study/data/total_extracted_result/all_cases.csv', encoding="utf-8")
# The lawyer inforamtion need to be corrected
all_cases = all_cases[~(all_cases['defendant']==all_cases['lawyer'])]
# all_cases.head()

In [7]:
all_cases[all_cases['plaintiff']=="王晓晖"]

,Unnamed: 0,case_id,judgement_date,is_success,court_name,procedure,judge,legalfee,objectmoney,province,city,reason,district,penalty,year,month,day,plaintiff,defendant,lawyer
60912,61172,5ee073843edbe71b2f7a01f1,2019-10-14,TRUE,山东省日照市中级人民法院,二审,林芳,NaN,0.0,山东省,日照市,服务合同纠纷,NaN,NaN,2019,10,14,王晓晖,日照华美酒店有限公司,赵鹏
60913,61173,5ee073843edbe71b2f7a01f1,2019-10-14,TRUE,山东省日照市中级人民法院,二审,林芳,NaN,0.0,山东省,日照市,服务合同纠纷,NaN,NaN,2019,10,14,王晓晖,日照华美酒店有限公司,李玲
60914,61174,5ee073843edbe71b2f7a01f1,2019-10-14,TRUE,山东省日照市中级人民法院,二审,林芳,NaN,0.0,山东省,日照市,服务合同纠纷,NaN,NaN,2019,10,14,王晓晖,日照华美酒店有限公司,高凤
60915,61175,5ee073843edbe71b2f7a01f1,2019-10-14,TRUE,山东省日照市中级人民法院,二审,林芳,NaN,0.0,山东省,日照市,服务合同纠纷,NaN,NaN,2019,10,14,王晓晖,郭长春,赵鹏
60916,61176,5ee073843edbe71b2f7a01f1,2019-10-14,TRUE,山东省日照市中级人民法院,二审,林芳,NaN,0.0,山东省,日照市,服务合同纠纷,NaN,NaN,2019,10,14,王晓晖,郭长春,李玲
60917,61177,5ee073843edbe71b2f7a01f1,2019-10-14,TRUE,山东省日照市中级人民法院,二审,林芳,NaN,0.0,山东省,日照市,服务合同纠纷,NaN,NaN,2019,10,14,王晓晖,郭长春,高凤


In [8]:
node_types = ['case', 'plaintiff', 'defendant', 'lawyer', 'court', 'judge']
edge_types = ["case_plaintiff", 
              "case_defendant", 
              "plaintiff_defendant", 
              "plaintiff_lawyer", 
              "case_court", 
              "case_judge", 
              "case_lawyer", 
              "judge_court"]

<h3>cases</h3>

In [9]:
cases = all_cases[['case_id', 
                   'judgement_date', 
                   'is_success', 
                   'procedure', 
                   'legalfee', 
                   'objectmoney', 
                   'province', 
                   'city', 
                   'district', 
                   'reason', 
                   'penalty']].drop_duplicates().reset_index()

cases['index'] = cases.index
cases['index'] = 'case_' + cases['index'].astype(str)
cases.rename(columns={"case_id": "case", "index": "case_id"}, inplace=True)
cases.head()

,case_id,case,judgement_date,is_success,procedure,legalfee,objectmoney,province,city,district,reason,penalty
0,case_0,57ab9058c2265c28a560195d,2014-01-13,TRUE,一审,244.0,374.48,浙江省,杭州市,西湖区,买卖合同纠纷,3744.8
1,case_1,57baba28c2265c5f452d2cef,2014-01-27,FALSE,一审,50.0,0.00,陕西省,安康市,汉滨区,买卖合同纠纷,0.0
2,case_2,57abb74ac2265c258984430a,2014-01-28,TRUE,一审,25.0,450.00,浙江省,嘉兴市,嘉善县,民事案由,450.0
3,case_3,57ad1fe7c2265c04d1192a41,2014-01-10,TRUE,一审,80.0,1312.00,重庆市,重庆市,沙坪坝区,产品责任纠纷,6560.0
4,case_4,57baba99c2265c5f452d3090,2014-01-27,FALSE,一审,50.0,0.00,陕西省,安康市,汉滨区,买卖合同纠纷,0.0


<h3>plaintiffs</h3>

In [10]:
plaintiffs = all_cases['plaintiff'].drop_duplicates().reset_index()
plaintiffs = plaintiffs.dropna()
plaintiffs['index'] = plaintiffs.index
plaintiffs['index'] = 'plaintiff_' + plaintiffs['index'].astype(str)
plaintiffs.rename(columns={"index": "plaintiff_id"}, inplace=True)
plaintiffs.head()

,plaintiff_id,plaintiff
0,plaintiff_0,喻忠
1,plaintiff_1,骆永明
2,plaintiff_2,白世桥
3,plaintiff_3,周巧
4,plaintiff_4,乔小锋


<h3>defendants</h3>

In [11]:
# defendants
defendants = all_cases['defendant'].drop_duplicates().reset_index()
defendants = defendants.dropna()
defendants['index'] = defendants.index
defendants['index'] = 'defendant_' + defendants['index'].astype(str)
defendants.rename(columns={"index": "defendant_id"}, inplace=True)
defendants.head()

,defendant_id,defendant
0,defendant_0,广东粤北行健康食品有限公司
1,defendant_1,浙江淘宝网络有限公司
2,defendant_2,冯坤
3,defendant_3,嘉善大润发商业有限公司
4,defendant_4,重庆和平药房连锁有限责任公司沙坪坝药妆店


<h3>lawyers</h3>

In [12]:
# lawyers
lawyers = all_cases['lawyer'].drop_duplicates().reset_index()
lawyers = lawyers.dropna()
lawyers['index'] = lawyers.index
lawyers['index'] = 'lawyer_' + lawyers['index'].astype(str)
lawyers.rename(columns={"index": "lawyer_id"}, inplace=True)
lawyers.head()

,lawyer_id,lawyer
1,lawyer_1,唐荣生
2,lawyer_2,赵鹏
3,lawyer_3,陈建华
4,lawyer_4,王桂苹
5,lawyer_5,赵亮


<h3>courts</h3>

In [13]:
# courts
courts = all_cases['court_name'].drop_duplicates().reset_index()
courts = courts.dropna()
courts['index'] = courts.index
courts['index'] = 'court_' + courts['index'].astype(str)
courts.rename(columns={"index": "court_id", "court_name": "court"}, inplace=True)
courts.head()

,court_id,court
0,court_0,杭州市西湖区人民法院
1,court_1,安康市汉滨区人民法院
2,court_2,嘉善县人民法院
3,court_3,重庆市沙坪坝区人民法院
4,court_4,厦门市思明区人民法院


<h3>judges</h3>

In [14]:
# judges
judges = all_cases['judge'].drop_duplicates().reset_index()
judges = judges.dropna()
judges['index'] = judges.index
judges['index'] = 'judge_' + judges['index'].astype(str)
judges.rename(columns={"index": "judge_id"}, inplace=True)
judges.head()

,judge_id,judge
0,judge_0,萧京
1,judge_1,张延安
2,judge_2,吕学强
3,judge_3,何思静
4,judge_4,邱瑛


In [15]:
all_cases["case_plaintiff"] = all_cases[['case_id', 'plaintiff']].apply(tuple, axis=1)
all_cases["case_defendant"] = all_cases[['case_id', 'defendant']].apply(tuple, axis=1)
all_cases["plaintiff_defendant"] = all_cases[['plaintiff', 'defendant']].apply(tuple, axis=1)
all_cases["plaintiff_lawyer"] = all_cases[['plaintiff', 'lawyer']].apply(tuple, axis=1)
all_cases["case_court"] = all_cases[['case_id', 'court_name']].apply(tuple, axis=1)
all_cases["case_judge"] = all_cases[['case_id', 'judge']].apply(tuple, axis=1)
all_cases["case_lawyer"] = all_cases[['case_id', 'lawyer']].apply(tuple, axis=1)
all_cases["judge_court"] = all_cases[['judge', 'court_name']].apply(tuple, axis=1)

<h3>case_plaintiff</h3>

In [16]:
# case_plaintiff
case_plaintiff = all_cases["case_plaintiff"].drop_duplicates().reset_index()
case_plaintiff['case'] = case_plaintiff['case_plaintiff'].apply(lambda row: row[0])
case_plaintiff['plaintiff'] = case_plaintiff['case_plaintiff'].apply(lambda row: row[1])
case_plaintiff['index'] = case_plaintiff.index
case_plaintiff['index'] = 'rcase_plaintiff_' + case_plaintiff['index'].astype(str)
case_plaintiff = case_plaintiff.dropna()
case_plaintiff.rename(columns={"index": "relation_id"}, inplace=True)

case_plaintiff = case_plaintiff.merge(cases[['case_id', 'case']], on="case", how="left").merge(plaintiffs, on="plaintiff", how="left")
case_plaintiff.head()

,relation_id,case_plaintiff,case,plaintiff,case_id,plaintiff_id
0,rcase_plaintiff_0,"(57ab9058c2265c28a560195d, 喻忠)",57ab9058c2265c28a560195d,喻忠,case_0,plaintiff_0
1,rcase_plaintiff_1,"(57baba28c2265c5f452d2cef, 骆永明)",57baba28c2265c5f452d2cef,骆永明,case_1,plaintiff_1
2,rcase_plaintiff_2,"(57abb74ac2265c258984430a, 白世桥)",57abb74ac2265c258984430a,白世桥,case_2,plaintiff_2
3,rcase_plaintiff_3,"(57ad1fe7c2265c04d1192a41, 周巧)",57ad1fe7c2265c04d1192a41,周巧,case_3,plaintiff_3
4,rcase_plaintiff_4,"(57baba99c2265c5f452d3090, 乔小锋)",57baba99c2265c5f452d3090,乔小锋,case_4,plaintiff_4


In [17]:
case_plaintiff[case_plaintiff['plaintiff']=="王晓晖"]

,relation_id,case_plaintiff,case,plaintiff,case_id,plaintiff_id
37285,rcase_plaintiff_37526,"(5ee073843edbe71b2f7a01f1, 王晓晖)",5ee073843edbe71b2f7a01f1,王晓晖,case_37392,plaintiff_5376


<h3>case_defendant</h3>

In [18]:
# case_defendant
case_defendant = all_cases["case_defendant"].drop_duplicates().reset_index()
case_defendant['case'] = case_defendant['case_defendant'].apply(lambda row: row[0])
case_defendant['defendant'] = case_defendant['case_defendant'].apply(lambda row: row[1])
case_defendant['index'] = case_defendant.index
case_defendant['index'] = 'rcase_defendant_' + case_defendant['index'].astype(str)
case_defendant = case_defendant.dropna()
case_defendant.rename(columns={"index": "relation_id"}, inplace=True)

case_defendant = case_defendant.merge(
    cases[['case_id', 'case']], on="case", how="left")\
.merge(defendants, on="defendant", how="left")
case_defendant.head()

,relation_id,case_defendant,case,defendant,case_id,defendant_id
0,rcase_defendant_0,"(57ab9058c2265c28a560195d, 广东粤北行健康食品有限公司)",57ab9058c2265c28a560195d,广东粤北行健康食品有限公司,case_0,defendant_0
1,rcase_defendant_1,"(57ab9058c2265c28a560195d, 浙江淘宝网络有限公司)",57ab9058c2265c28a560195d,浙江淘宝网络有限公司,case_0,defendant_1
2,rcase_defendant_2,"(57baba28c2265c5f452d2cef, 冯坤)",57baba28c2265c5f452d2cef,冯坤,case_1,defendant_2
3,rcase_defendant_3,"(57abb74ac2265c258984430a, 嘉善大润发商业有限公司)",57abb74ac2265c258984430a,嘉善大润发商业有限公司,case_2,defendant_3
4,rcase_defendant_4,"(57ad1fe7c2265c04d1192a41, 重庆和平药房连锁有限责任公司沙坪坝药妆店)",57ad1fe7c2265c04d1192a41,重庆和平药房连锁有限责任公司沙坪坝药妆店,case_3,defendant_4


<h3>plaintiff_defendant</h3>

In [19]:
# plaintiff_defendant
plaintiff_defendant = all_cases["plaintiff_defendant"].drop_duplicates().reset_index()
plaintiff_defendant['plaintiff'] = plaintiff_defendant['plaintiff_defendant'].apply(lambda row: row[0])
plaintiff_defendant['defendant'] = plaintiff_defendant['plaintiff_defendant'].apply(lambda row: row[1])
plaintiff_defendant['index'] = plaintiff_defendant.index
plaintiff_defendant['index'] = 'rplaintiff_defendant_' + plaintiff_defendant['index'].astype(str)
plaintiff_defendant = plaintiff_defendant.dropna()
plaintiff_defendant.rename(columns={"index": "relation_id"}, inplace=True)

plaintiff_defendant = plaintiff_defendant.merge(
    plaintiffs, on="plaintiff", how="left")\
.merge(defendants, on="defendant", how="left")
plaintiff_defendant.head()

,relation_id,plaintiff_defendant,plaintiff,defendant,plaintiff_id,defendant_id
0,rplaintiff_defendant_0,"(喻忠, 广东粤北行健康食品有限公司)",喻忠,广东粤北行健康食品有限公司,plaintiff_0,defendant_0
1,rplaintiff_defendant_1,"(喻忠, 浙江淘宝网络有限公司)",喻忠,浙江淘宝网络有限公司,plaintiff_0,defendant_1
2,rplaintiff_defendant_2,"(骆永明, 冯坤)",骆永明,冯坤,plaintiff_1,defendant_2
3,rplaintiff_defendant_3,"(白世桥, 嘉善大润发商业有限公司)",白世桥,嘉善大润发商业有限公司,plaintiff_2,defendant_3
4,rplaintiff_defendant_4,"(周巧, 重庆和平药房连锁有限责任公司沙坪坝药妆店)",周巧,重庆和平药房连锁有限责任公司沙坪坝药妆店,plaintiff_3,defendant_4


<h3>plaintiff_lawyer</h3>

In [20]:
# plaintiff_lawyer
plaintiff_lawyer = all_cases["plaintiff_lawyer"].drop_duplicates().reset_index()
plaintiff_lawyer['plaintiff'] = plaintiff_lawyer['plaintiff_lawyer'].apply(lambda row: row[0])
plaintiff_lawyer['lawyer'] = plaintiff_lawyer['plaintiff_lawyer'].apply(lambda row: row[1])
plaintiff_lawyer['index'] = plaintiff_lawyer.index
plaintiff_lawyer['index'] = 'rcplaintiff_lawyer_' + plaintiff_lawyer['index'].astype(str)
plaintiff_lawyer = plaintiff_lawyer.dropna()
plaintiff_lawyer.rename(columns={"index": "relation_id"}, inplace=True)

plaintiff_lawyer = plaintiff_lawyer.merge(
    plaintiffs, on="plaintiff", how="left")\
.merge(lawyers, on="lawyer", how="left")
plaintiff_lawyer.head()

,relation_id,plaintiff_lawyer,plaintiff,lawyer,plaintiff_id,lawyer_id
0,rcplaintiff_lawyer_1,"(骆永明, 唐荣生)",骆永明,唐荣生,plaintiff_1,lawyer_1
1,rcplaintiff_lawyer_4,"(乔小锋, 唐荣生)",乔小锋,唐荣生,plaintiff_4,lawyer_1
2,rcplaintiff_lawyer_6,"(赵鹏, 赵鹏)",赵鹏,赵鹏,plaintiff_6,lawyer_2
3,rcplaintiff_lawyer_8,"(陈慧群, 陈建华)",陈慧群,陈建华,plaintiff_8,lawyer_3
4,rcplaintiff_lawyer_10,"(赵德明, 王桂苹)",赵德明,王桂苹,plaintiff_10,lawyer_4


<h3>case_court</h3>

In [21]:
# case_court
case_court = all_cases["case_court"].drop_duplicates().reset_index()
case_court['case'] = case_court['case_court'].apply(lambda row: row[0])
case_court['court'] = case_court['case_court'].apply(lambda row: row[1])
case_court['index'] = case_court.index
case_court['index'] = 'rcase_court_' + case_court['index'].astype(str)
case_court = case_court.dropna()
case_court.rename(columns={"index": "relation_id"}, inplace=True)

case_court = case_court.merge(
    cases[['case_id', 'case']], on="case", how="left").merge(courts, on="court", how="left")
case_court.head()

,relation_id,case_court,case,court,case_id,court_id
0,rcase_court_0,"(57ab9058c2265c28a560195d, 杭州市西湖区人民法院)",57ab9058c2265c28a560195d,杭州市西湖区人民法院,case_0,court_0
1,rcase_court_1,"(57baba28c2265c5f452d2cef, 安康市汉滨区人民法院)",57baba28c2265c5f452d2cef,安康市汉滨区人民法院,case_1,court_1
2,rcase_court_2,"(57abb74ac2265c258984430a, 嘉善县人民法院)",57abb74ac2265c258984430a,嘉善县人民法院,case_2,court_2
3,rcase_court_3,"(57ad1fe7c2265c04d1192a41, 重庆市沙坪坝区人民法院)",57ad1fe7c2265c04d1192a41,重庆市沙坪坝区人民法院,case_3,court_3
4,rcase_court_4,"(57baba99c2265c5f452d3090, 安康市汉滨区人民法院)",57baba99c2265c5f452d3090,安康市汉滨区人民法院,case_4,court_1


<h3>case_judge</h3>

In [22]:
# case_judge
case_judge = all_cases["case_judge"].drop_duplicates().reset_index()
case_judge['case'] = case_judge['case_judge'].apply(lambda row: row[0])
case_judge['judge'] = case_judge['case_judge'].apply(lambda row: row[1])
case_judge['index'] = case_judge.index
case_judge['index'] = 'rcase_judge_' + case_judge['index'].astype(str)
case_judge = case_judge.dropna()
case_judge.rename(columns={"index": "relation_id"}, inplace=True)

case_judge = case_judge.merge(
    cases[['case_id', 'case']], on="case", how="left").merge(judges, on="judge", how="left")
case_judge.head()

,relation_id,case_judge,case,judge,case_id,judge_id
0,rcase_judge_0,"(57ab9058c2265c28a560195d, 萧京)",57ab9058c2265c28a560195d,萧京,case_0,judge_0
1,rcase_judge_1,"(57baba28c2265c5f452d2cef, 张延安)",57baba28c2265c5f452d2cef,张延安,case_1,judge_1
2,rcase_judge_2,"(57abb74ac2265c258984430a, 吕学强)",57abb74ac2265c258984430a,吕学强,case_2,judge_2
3,rcase_judge_3,"(57ad1fe7c2265c04d1192a41, 何思静)",57ad1fe7c2265c04d1192a41,何思静,case_3,judge_3
4,rcase_judge_4,"(57baba99c2265c5f452d3090, 张延安)",57baba99c2265c5f452d3090,张延安,case_4,judge_1


<h3>case_lawyer</h3>

In [23]:
# case_lawyer
case_lawyer = all_cases["case_lawyer"].drop_duplicates().reset_index()
case_lawyer['case'] = case_lawyer['case_lawyer'].apply(lambda row: row[0])
case_lawyer['lawyer'] = case_lawyer['case_lawyer'].apply(lambda row: row[1])
case_lawyer['index'] = case_lawyer.index
case_lawyer['index'] = 'rcase_lawyer_' + case_lawyer['index'].astype(str)
case_lawyer = case_lawyer.dropna()
case_lawyer.rename(columns={"index": "relation_id"}, inplace=True)

case_lawyer = case_lawyer.merge(
    cases[['case_id', 'case']], on="case", how="left").merge(lawyers, on="lawyer", how="left")
case_lawyer.head()

,relation_id,case_lawyer,case,lawyer,case_id,lawyer_id
0,rcase_lawyer_1,"(57baba28c2265c5f452d2cef, 唐荣生)",57baba28c2265c5f452d2cef,唐荣生,case_1,lawyer_1
1,rcase_lawyer_4,"(57baba99c2265c5f452d3090, 唐荣生)",57baba99c2265c5f452d3090,唐荣生,case_4,lawyer_1
2,rcase_lawyer_6,"(57baba8fc2265c5f452d303f, 唐荣生)",57baba8fc2265c5f452d303f,唐荣生,case_6,lawyer_1
3,rcase_lawyer_8,"(57ac901ec2265c28a56a8c9a, 赵鹏)",57ac901ec2265c28a56a8c9a,赵鹏,case_8,lawyer_2
4,rcase_lawyer_10,"(57baba88c2265c5f452d2ff2, 唐荣生)",57baba88c2265c5f452d2ff2,唐荣生,case_10,lawyer_1


<h3>judge_court</h3>

In [24]:
# judge_court
judge_court = all_cases["judge_court"].drop_duplicates().reset_index()
judge_court['judge'] = judge_court['judge_court'].apply(lambda row: row[0])
judge_court['court'] = judge_court['judge_court'].apply(lambda row: row[1])
judge_court['index'] = judge_court.index
judge_court['index'] = 'rjudge_court_' + judge_court['index'].astype(str)
judge_court = judge_court.dropna()
judge_court.rename(columns={"index": "relation_id"}, inplace=True)

judge_court = judge_court.merge(
    judges, on="judge", how="left").merge(courts, on="court", how="left")
judge_court.head()

,relation_id,judge_court,judge,court,judge_id,court_id
0,rjudge_court_0,"(萧京, 杭州市西湖区人民法院)",萧京,杭州市西湖区人民法院,judge_0,court_0
1,rjudge_court_1,"(张延安, 安康市汉滨区人民法院)",张延安,安康市汉滨区人民法院,judge_1,court_1
2,rjudge_court_2,"(吕学强, 嘉善县人民法院)",吕学强,嘉善县人民法院,judge_2,court_2
3,rjudge_court_3,"(何思静, 重庆市沙坪坝区人民法院)",何思静,重庆市沙坪坝区人民法院,judge_3,court_3
4,rjudge_court_4,"(邱瑛, 厦门市思明区人民法院)",邱瑛,厦门市思明区人民法院,judge_4,court_4


<h2>Output all data files for neo4j desktop use.</h2>

In [26]:
cases.to_csv("/Users/starice/Desktop/neo4j_case_data/cases.csv", encoding="utf-8")
plaintiffs.to_csv("/Users/starice/Desktop/neo4j_case_data/plaintiffs.csv", encoding="utf-8")
defendants.to_csv("/Users/starice/Desktop/neo4j_case_data/defendants.csv", encoding="utf-8")
courts.to_csv("/Users/starice/Desktop/neo4j_case_data/courts.csv", encoding="utf-8")
lawyers.to_csv("/Users/starice/Desktop/neo4j_case_data/lawyers.csv", encoding="utf-8")
judges.to_csv("/Users/starice/Desktop/neo4j_case_data/judges.csv", encoding="utf-8")

In [27]:
case_plaintiff.to_csv("/Users/starice/Desktop/neo4j_case_data/case_plaintiff.csv", encoding="utf-8")
case_defendant.to_csv("/Users/starice/Desktop/neo4j_case_data/case_defendant.csv", encoding="utf-8")
plaintiff_defendant.to_csv("/Users/starice/Desktop/neo4j_case_data/plaintiff_defendant.csv", encoding="utf-8")
plaintiff_lawyer.to_csv("/Users/starice/Desktop/neo4j_case_data/plaintiff_lawyer.csv", encoding="utf-8")
case_court.to_csv("/Users/starice/Desktop/neo4j_case_data/case_court.csv", encoding="utf-8")
case_judge.to_csv("/Users/starice/Desktop/neo4j_case_data/case_judge.csv", encoding="utf-8")
case_lawyer.to_csv("/Users/starice/Desktop/neo4j_case_data/case_lawyer.csv", encoding="utf-8")
judge_court.to_csv("/Users/starice/Desktop/neo4j_case_data/judge_court.csv", encoding="utf-8")